# Associate Rule Mining (ARM) on Workforce Diveristy Outcomes Record Dataset

## 1. Introduction

Associate rule mining (ARM) is concerned with associations between data items. Since ARM is unsupervised, we are most concerned with the feature variables, or X. For my record dataset, the X variables are the categorical variables seniority, job category, gender, and ethnicity, as well as the continuous variables salary, outflow, inflow, and count. I will verify the accuracy of each model by comparing the predicted clusters and target company labels. Through my ARM analysis, I am hoping to discover relationships between my X variables.

## 2. Theory

## 3. Methods

## 4. Results

## 5. Conclusion



## 6. References